# Práctica 1C - Inteligencia Artificial
## Belén Díaz Agudo -  Facultad de Informática UCM
## Búsqueda local
En esta primera parte usaremos ejercicios paso a paso para familiarizarnos con la resolución de problemas sencillos de optimización, como la maximización o minimización de una función, o el problema de la mochila o del viajante, problemas conocidos cuya resolución se ha abordado con técnicas algorítmicas y que vamos a resolver utilizando algoritmos de búsqueda local. En la segunda parte de la práctica se pide resolver el problema de la organización de jornadas informáticas dado en el enunciado.

## Parte 1. Algoritmo de escalada
Hill Climbing es un algoritmo de búsqueda local heurística utilizada para problemas de optimización.
Esta solución puede o no ser el óptimo global. El algoritmo es una variante del algoritmo de generación y prueba.
<br>
En general, el algoritmo funciona de la siguiente manera:
- Evaluar el estado inicial.
- Si es igual al estado del objetivo, terminamos.
- Encuentra un estado vecino al estado actual
- Evaluar este estado. Si está más cerca del estado objetivo que antes, reemplace el estado inicial con este estado y repita estos pasos.
<br>
Usaremos la implementación de AIMA que está en el módulo search.py

    def hill_climbing(problem):
        """From the initial node, keep choosing the neighbor with highest value,
        stopping when no neighbor is better. [Figure 4.2]"""
        current = Node(problem.initial)
        while True:
            neighbors = current.expand(problem)
            if not neighbors:
                break
            neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
            if problem.value(neighbor.state) <= problem.value(current.state):
                break
            current = neighbor
        return current.state


### TSP (Travelling Salesman Problem): el problema del viajante
Dado un conjunto de ciudades y la distancia entre cada par de ciudades, el problema es encontrar la ruta más corta posible que visite cada ciudad exactamente una vez y regrese al punto de partida. Es un problema NP hard. No existen una solución de coste polinomial. 

In [33]:
##Resolvereremos el problema del viajante TSP para encontrar una solución aproximada.
from search import *

class TSP_problem(Problem):

    def two_opt(self, state):
        """ Neighbour generating function for Traveling Salesman Problem """
        neighbour_state = state[:]
        left = random.randint(0, len(neighbour_state) - 1)
        right = random.randint(0, len(neighbour_state) - 1)
        if left > right:
            left, right = right, left
        neighbour_state[left: right + 1] = reversed(neighbour_state[left: right + 1])
        return neighbour_state

    def actions(self, state):
        """ action that can be excuted in given state """
        return [self.two_opt]

    def result(self, state, action):
        """  result after applying the given action on the given state """
        return action(state)

    def path_cost(self, c, state1, action, state2):
        """ total distance for the Traveling Salesman to be covered if in state2  """
        cost = 0
        for i in range(len(state2) - 1):
            cost += distances[state2[i]][state2[i + 1]]
        cost += distances[state2[0]][state2[-1]]
        return cost

    def value(self, state):
        """ value of path cost given negative for the given state """
        return -1 * self.path_cost(None, None, None, state)

In [34]:
## Resolveremos el TSP para las ciudades de la lista de ciudades de Rumanía.
## ['Arad', 'Bucharest', 'Craiova', 'Drobeta', 'Eforie', 'Fagaras', 'Giurgiu', 'Hirsova', 'Iasi', 'Lugoj', 'Mehadia', 'Neamt', 'Oradea', 'Pitesti', 'Rimnicu', 'Sibiu', 'Timisoara', 'Urziceni', 'Vaslui', 'Zerind']

In [35]:
# Usaremos la siguiente representacion del libro AIMA para el mapa de Rumanía.

romania_map = UndirectedGraph(dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142)))

romania_map.locations = dict(
    Arad=(91, 492), Bucharest=(400, 327), Craiova=(253, 288),
    Drobeta=(165, 299), Eforie=(562, 293), Fagaras=(305, 449),
    Giurgiu=(375, 270), Hirsova=(534, 350), Iasi=(473, 506),
    Lugoj=(165, 379), Mehadia=(168, 339), Neamt=(406, 537),
    Oradea=(131, 571), Pitesti=(320, 368), Rimnicu=(233, 410),
    Sibiu=(207, 457), Timisoara=(94, 410), Urziceni=(456, 350),
    Vaslui=(509, 444), Zerind=(108, 531))

Es bastante sencillo entender este `romania_map`. El primer nodo ** Arad ** tiene tres vecinos llamados ** Zerind **, ** Sibiu **, ** Timisoara **. Cada uno de estos nodos son 75, 140, 118 unidades aparte de ** Arad ** respectivamente. Y lo mismo ocurre con otros nodos.

Y `romania_map.locations` contiene las posiciones de cada uno de los nodos. 
Como heurística se puede usar la distancia en línea recta o la distancia manhattan (que es diferente de la proporcionada en `romania_map`) entre dos ciudades.

In [36]:
romania_locations = romania_map.locations
print(romania_locations)

{'Arad': (91, 492), 'Bucharest': (400, 327), 'Craiova': (253, 288), 'Drobeta': (165, 299), 'Eforie': (562, 293), 'Fagaras': (305, 449), 'Giurgiu': (375, 270), 'Hirsova': (534, 350), 'Iasi': (473, 506), 'Lugoj': (165, 379), 'Mehadia': (168, 339), 'Neamt': (406, 537), 'Oradea': (131, 571), 'Pitesti': (320, 368), 'Rimnicu': (233, 410), 'Sibiu': (207, 457), 'Timisoara': (94, 410), 'Urziceni': (456, 350), 'Vaslui': (509, 444), 'Zerind': (108, 531)}


In [37]:
# node colors, node positions and node label positions
node_colors = {node: 'white' for node in romania_map.locations.keys()}
node_positions = romania_map.locations
node_label_pos = { k:[v[0],v[1]-10]  for k,v in romania_map.locations.items() }
edge_weights = {(k, k2) : v2 for k, v in romania_map.graph_dict.items() for k2, v2 in v.items()}

romania_graph_data = {  'graph_dict' : romania_map.graph_dict,
                        'node_colors': node_colors,
                        'node_positions': node_positions,
                        'node_label_positions': node_label_pos,
                         'edge_weights': edge_weights
                     }

In [38]:
from notebook import show_map, final_path_colors, display_visual
import matplotlib

show_map(romania_graph_data)

ImportError: cannot import name 'show_map' from 'notebook' (C:\Users\Usuario\anaconda3\lib\site-packages\notebook\__init__.py)

In [39]:
## el siguiente código crea un diccionario y calcula y añade al diccionario la distancia manhattan entre las ciudades. 
import numpy as np

distances = {}
all_cities = []

for city in romania_map.locations.keys():
    distances[city] = {}
    all_cities.append(city)
    
all_cities.sort()
print(all_cities)

for name_1, coordinates_1 in romania_map.locations.items():
        for name_2, coordinates_2 in romania_map.locations.items():
            distances[name_1][name_2] = np.linalg.norm(
                [coordinates_1[0] - coordinates_2[0], coordinates_1[1] - coordinates_2[1]])
            distances[name_2][name_1] = np.linalg.norm(
                [coordinates_1[0] - coordinates_2[0], coordinates_1[1] - coordinates_2[1]])

['Arad', 'Bucharest', 'Craiova', 'Drobeta', 'Eforie', 'Fagaras', 'Giurgiu', 'Hirsova', 'Iasi', 'Lugoj', 'Mehadia', 'Neamt', 'Oradea', 'Pitesti', 'Rimnicu', 'Sibiu', 'Timisoara', 'Urziceni', 'Vaslui', 'Zerind']


In [40]:
# Creamos una instancia del problema TSP con la lista de ciudades anterior que se na extraido del mapa.
# En el mapa hay informacion de las distancias que se utilizan en la clase TSP_problem para calcular el coste y las heurísticas.
tsp = TSP_problem(all_cities)

In [41]:
## Redefinimos el hill climbing de AIMA para que el método de generacion de vecinos sea acceder al grafo que hemos definido para el TSP

def hill_climbing(problem):
    
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2]"""
    
    def find_neighbors(state, number_of_neighbors=100):
        """ finds neighbors using two_opt method """
        
        neighbors = []
        
        for i in range(number_of_neighbors):
            new_state = problem.two_opt(state)
            neighbors.append(Node(new_state))
            state = new_state
            
        return neighbors

    # as this is a stochastic algorithm, we will set a cap on the number of iterations
    iterations = 10000
    
    current = Node(problem.initial)
    while iterations:
        neighbors = find_neighbors(current.state)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) >= problem.value(current.state):
            current.state = neighbor.state
        iterations -= 1
        
    return current.state

In [46]:
# Y lo resolvemos con escalada. 
solucion1 = hill_climbing(tsp)
solucion1

['Rimnicu',
 'Fagaras',
 'Neamt',
 'Iasi',
 'Vaslui',
 'Hirsova',
 'Eforie',
 'Urziceni',
 'Bucharest',
 'Giurgiu',
 'Pitesti',
 'Craiova',
 'Drobeta',
 'Mehadia',
 'Lugoj',
 'Timisoara',
 'Arad',
 'Zerind',
 'Oradea',
 'Sibiu']

In [54]:
# Y lo resolvemos con escalada. 
solucion2 = hill_climbing(tsp)
solucion2

['Fagaras',
 'Neamt',
 'Iasi',
 'Vaslui',
 'Hirsova',
 'Eforie',
 'Urziceni',
 'Bucharest',
 'Giurgiu',
 'Pitesti',
 'Craiova',
 'Drobeta',
 'Mehadia',
 'Lugoj',
 'Timisoara',
 'Arad',
 'Zerind',
 'Oradea',
 'Sibiu',
 'Rimnicu']

In [52]:
# Y lo resolvemos con escalada. 
solucion3 = hill_climbing(tsp)
solucion3

['Timisoara',
 'Mehadia',
 'Drobeta',
 'Craiova',
 'Giurgiu',
 'Bucharest',
 'Urziceni',
 'Eforie',
 'Hirsova',
 'Vaslui',
 'Iasi',
 'Neamt',
 'Fagaras',
 'Pitesti',
 'Rimnicu',
 'Lugoj',
 'Sibiu',
 'Oradea',
 'Zerind',
 'Arad']

In [55]:
print("Distancia de la solución 1: " + str(tsp.path_cost(None, None, None, solucion1)))
print("Distancia de la solución 2: " + str(tsp.path_cost(None, None, None, solucion2)))
print("Distancia de la solución 3: " + str(tsp.path_cost(None, None, None, solucion3)))

Distancia de la solución 1: 1589.8433308050924
Distancia de la solución 2: 1589.8433308050924
Distancia de la solución 3: 1688.112533462133


### Ejercicio 1. Resuelve el problema TSP con el algoritmo de escalada por máxima pendiente en el mapa de ciudades de Rumanía y explica el resultado obtenido. 

Realiza un análisis razonado de las propiedades del algoritmo: eficiencia y optimalidad en base a la ejecución. 

¿Ha encontrado el algoritmo el óptimo global? 
¿Ha encontrado la misma solución en distintas ejecuciones?

Sólo se pide hacer una comparativa teórica (breve) con cómo se comporta este algoritmo y relacionarlo con otros algoritmos vistos en clase. 

Opcionalmente se puede hacer la comparativa real con algún algoritmo de búsqueda exhaustiva. 

#### Comentario

Primero de todo, vemos que en las tres soluciones que hemos obtenido, las ciudades obtenidas no son necesariamente adyacentes ni forman un ciclo. 

Además, vemos que para un mismo estado inicial, el algoritmo de escalada por máxima pendiente nos ha dado soluciones distintas en las tres llamadas que hemos hecho al algoritmo, en las que además una de ellas tiene un coste distinto. Por ello se ve claro que este algoritmo no encuentra necesariamente el óptimo global, ya las tres soluciones tienen un valor distinto y perfectamente podría haber otra solución más óptima que las obtenidas.

Que el algoritmo encuentre distintas soluciones en distintas ejecuciones debe deberse a que se generan solo algunos de los nodos vecinos de forma aleatoria, lo que daría lugar a que el algoritmo encuentre distintas soluciones en cada ejecución, aun teniendo el mismo estado inicial.

La principal diferencia entre este tipo de algoritmos y otros algoritmos como los de búsqueda ciega y heurística, es el no determinsmo. En búsqueda ciega aseguramos recorrer todo o gran parte del árbol de búsqueda. En búsqueda heurística podemos descartar nodos y solo explorar los que se consideran más prometedores, ahorrándonos explorar parte del árbol. En ambas gastamos más tiempo de ejecución, pero nos garantizan llegar a una solución óptima. En búsqueda local, sin embargo, ahorramos tiempo de ejcución a cambio de no garantizar de llegar a la solución óptima. Simplemente queda esperar que nos lleven a una solución que consideremos suficientemente buena.

## Parte 2. Enfriamiento simulado ( simulated annealing) 
El algoritmo de enfriamiento simulado puede manejar las situaciones de óptimo local o mesetas típicas en algoritmos de escalada.
<br>
El enfriamiento simulado es bastante similar a la escalada pero en lugar de elegir el mejor movimiento en cada iteración, elige un movimiento aleatorio. Si este movimiento aleatorio nos acerca al óptimo global, será aceptado,
pero si no lo hace, el algoritmo puede aceptar o rechazar el movimiento en función de una probabilidad dictada por la temperatura.  Cuando la `temperatura` es alta, es más probable que el algoritmo acepte un movimiento aleatorio incluso si es malo. A bajas temperaturas, solo se aceptan buenos movimientos, con alguna excepción ocasional.
Esto permite la exploración del espacio de estado y evita que el algoritmo se atasque en el óptimo local.

    Usaremos la implementación de AIMA del modulo search.py
    
    def simulated_annealing(problem, schedule=exp_schedule()):
    """[Figure 4.5] CAUTION: This differs from the pseudocode as it
    returns a state instead of a Node."""
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return current.state
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0 or probability(math.exp(delta_e / T)):
            current = next_choice

Como hemos visto en clase hay varios métodos de enfriamiento (scheduling routine) 
Se puede variar el método de enfriamiento. En la implementación actual estamos usando el método de enfriamiento exponencial (que se pasa como parámetro). 

    def exp_schedule(k=20, lam=0.005, limit=100):
        """One possible schedule function for simulated annealing"""
        return lambda t: (k * math.exp(-lam * t) if t < limit else 0)

Como ejemplo, vamos a definir un problema sencillo de encontrar el punto más alto en una rejilla. Este problema está definido en el módulo search.py como PeakFindingProblem. Lo reproducimos aquí y creamos una rejilla simple.

In [109]:
initial = (0, 0)
grid = [[3, 7, 2, 8], [5, 2, 9, 1], [5, 3, 3, 1]]

In [110]:
# Pre-defined actions for PeakFindingProblem
directions4 = { 'W':(-1, 0), 'N':(0, 1), 'E':(1, 0), 'S':(0, -1) }
directions8 = dict(directions4) 
directions8.update({'NW':(-1, 1), 'NE':(1, 1), 'SE':(1, -1), 'SW':(-1, -1) })

class PeakFindingProblem(Problem):
    """Problem of finding the highest peak in a limited grid"""

    def __init__(self, initial, grid, defined_actions=directions4):
        """The grid is a 2 dimensional array/list whose state is specified by tuple of indices"""
        Problem.__init__(self, initial)
        self.grid = grid
        self.defined_actions = defined_actions
        self.n = len(grid)
        assert self.n > 0
        self.m = len(grid[0])
        assert self.m > 0

    def actions(self, state):
        """Returns the list of actions which are allowed to be taken from the given state"""
        allowed_actions = []
        for action in self.defined_actions:
            next_state = vector_add(state, self.defined_actions[action])
            if next_state[0] >= 0 and next_state[1] >= 0 and next_state[0] <= self.n - 1 and next_state[1] <= self.m - 1:
                allowed_actions.append(action)

        return allowed_actions

    def result(self, state, action):
        """Moves in the direction specified by action"""
        return vector_add(state, self.defined_actions[action])

    def value(self, state):
        """Value of a state is the value it is the index to"""
        x, y = state
        assert 0 <= x < self.n
        assert 0 <= y < self.m
        return self.grid[x][y]


In [111]:
problem = PeakFindingProblem(initial, grid, directions4)

In [112]:
# Lo resolvemos con enfriamiento simulado

solutions = {problem.value(simulated_annealing(problem)) for i in range(100)}
max(solutions)

9

In [113]:
def hill_climbing(problem):
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2]"""
    current = Node(problem.initial)
    while True:
        neighbors = current.expand(problem)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
            break
        current = neighbor
    return current.state

In [115]:
solution = problem.value(hill_climbing(problem))
solution

7

### Ejercicio 2.  Resuelve el problema anterior de encontrar el punto máximo en una rejilla. Comenta y razona los resultados obtenidos en distintas rejjillas con los algoritmos de enfriamiento simulado y escalada por máxima pendiente. 
 
 
Ejemplo de rejilla para pruebas

grid = [[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.40, 0.40, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 11.2, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 8.50, 4.30, 1.80, 0.70, 0.00, 0.00]]


### Grid1

In [149]:
grid1 = [[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.40, 0.40, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 11.2, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 8.50, 4.30, 1.80, 0.70, 0.00, 0.00]]


In [150]:
problem = PeakFindingProblem(initial, grid1, directions4)
solutions = {problem.value(simulated_annealing(problem)) for i in range(100)}
max(solutions)

11.2

In [151]:
problem.value(hill_climbing(problem))

0.0

En esta rejilla nuestro nodo inicial se encuentra en un terrerno llano. Por ello el aloritmo de escalada no consigue salir de ella (todos los vecinos tienen mismo valor que él) y por eso nos devuelve 0. En cambio, podemos apreciar que el algoritmo de enfriamiento sí que logra llegar al óptimo global.

### Grid2

In [161]:
grid2 = [[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.20, 0.20, 0.20, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.20, 0.20, 0.20, 0.40, 0.40, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 11.2, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 8.50, 4.30, 1.80, 0.70, 0.00, 0.00]]

In [164]:
problem = PeakFindingProblem(initial, grid2, directions4)
solutions = {problem.value(simulated_annealing(problem)) for i in range(100)}
max(solutions)

8.5

In [170]:
problem = PeakFindingProblem(initial, grid2, directions4)
solutions = {problem.value(simulated_annealing(problem)) for i in range(100)}
max(solutions)

11.2

In [169]:
problem.value(hill_climbing(problem))

0.2

En este caso vemos que, como el algoritmo de enfriamiento puede de forma aleatoria dada una probabilidad moverse a un nodo aunque este no mejore la solución, llegamos a una solución que no es la óptima global. Podemos ver en la segunda ejecución de ese mismo algoritmo que sí que llega al óptimo.

Por otro lado, del mismo modo que en el grid1, el algoritmo de escalada de máxima pendiente queda atascado si el nodo está rodeado por otros con el mismo valor ya que ninguno mejora la solución, por lo que nos da 0.2 como solución.

### Grid3

In [229]:
grid3 = [[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.20, 0.20, 0.20, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.30, 0.30, 0.30, 0.40, 0.40, 0.00, 0.00, 0.00, 0.00],
        [0.50, 0.50, 0.60, 0.70, 0.80, 0.90, 0.00, 0.70, 1.40],
        [2.20, 3.30, 4.00, 0.70, 0.70, 0.70, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 7.30, 7.80, 9.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 7.30, 7.80, 0.00, 11.2, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 9.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 6.50, 4.30, 1.80, 0.70, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 1.40],
        [2.20, 1.80, 0.70, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [2.20, 1.80, 4.70, 8.50, 4.30, 1.80, 0.70, 0.00, 0.00]]

In [236]:
problem = PeakFindingProblem(initial, grid3, directions4)
solutions = {problem.value(simulated_annealing(problem)) for i in range(100)}
max(solutions)

11.2

In [237]:
problem = PeakFindingProblem(initial, grid3, directions4)
solutions = {problem.value(simulated_annealing(problem)) for i in range(100)}
max(solutions)

9.7

In [232]:
problem.value(hill_climbing(problem))

11.2

In [233]:
problem.value(hill_climbing(problem))

7.8

Aquí vemos que si establecemos un camino que lleve al máximo, el algoritmo de máxima pendiente sí que logra llegar al óptimo global, aunque igualmente puede llegar a una solución no óptima. 

Del mismo modo que hemos visto antes, como el algoritmo de enfriamiento simulado puede visitar nodos que no mejoran la solución, podemos llegar a soluciónes que no son el óptimo global.

## Parte 3. Algoritmos genéticos


Se define una clase ProblemaGenetico que incluye los elementos necesarios para la representación de un problema de optimización que se va a resolver con un algoritmo genético. Los elementos son los que hemos visto en clase:

 - genes: lista de genes usados en el genotipo de los estados.
 - longitud_individuos: longitud de los cromosomas
 - decodifica: función de obtiene el fenotipo a partir del genotipo.
 - fitness: función de valoración.
 - muta: función de mutación de un cromosoma 
 - cruza: función de cruce de un par de cromosomas

In [111]:
import random

In [119]:
class ProblemaGenetico(object):
        def __init__(self, genes,fun_dec,fun_muta , fun_cruza, fun_fitness,longitud_individuos):
            self.genes = genes
            self.fun_dec = fun_dec
            self.fun_cruza = fun_cruza
            self.fun_muta = fun_muta
            self.fun_fitness = fun_fitness
            self.longitud_individuos = longitud_individuos
            """Constructor de la clase"""
                
        def decodifica(self, genotipo):
            """Devuelve el fenotipo a partir del genotipo"""
            fenotipo = self.fun_dec(genotipo)
            return fenotipo
        def muta(self, cromosoma,prob):
            """Devuelve el cromosoma mutado"""   
            mutante = self.fun_muta(cromosoma,prob)
            return mutante
        def cruza(self, cromosoma1, cromosoma2):         
            """Devuelve el cruce de un par de cromosomas"""
            cruce = self.fun_cruza(cromosoma1,cromosoma2)
            return cruce 
        def fitness(self, cromosoma):    
            """Función de valoración"""
            valoracion = self.fun_fitness(cromosoma)
            return valoracion

En primer lugar vamos a definir una instancia de la clase anterior correspondiente al problema de optimizar (maximizar o minimizar) la función cuadrado x^2 en el conjunto de los números naturales menores que 2^{10}.
Se usa este ejemplo (del que sabemos la solución) para ver todos los elementos y poder observar el comportamiento. 


In [3]:
# Será necesaria la siguiente función que interpreta una lista de 0's y 1's como un número natural:  
# La siguiente función que interpreta una lista de 0's y 1's como
# un número natural:  

def binario_a_decimal(x):
    return sum(b*(2**i) for (i,b) in enumerate(x)) 

In [4]:
list(enumerate([1, 0, 0]))

[(0, 1), (1, 0), (2, 0)]

In [4]:
# En primer luegar usaremos la clase anterior para representar el problema de optimizar (maximizar o minimizar)
# la función cuadrado en el conjunto de los números naturales menores que
# 2^{10}.

# Tenemos que definir funciones de cruce, mutación y fitness para este problema.

def fun_cruzar(cromosoma1, cromosoma2):
    """Cruza los cromosomas por la mitad"""
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    cruce1 = cromosoma1[0:l1//2]+cromosoma2[l1//2:l2]
    cruce2 = cromosoma2[0:l2//2]+cromosoma1[l2//2:l1]
    return [cruce1,cruce2]

def fun_mutar(cromosoma,prob):
    """Elige un elemento al azar del cromosoma y lo modifica con una probabilidad igual a prob"""
    l = len(cromosoma)
    p = random.randint(0,l-1)
    if prob > random.uniform(0,1):
        cromosoma[p] =  (cromosoma[p]+1)%2
    return cromosoma

def fun_fitness_cuad(cromosoma):
    """Función de valoración que eleva al cuadrado el número recibido en binario"""
    n = binario_a_decimal(cromosoma)**2
    return n

cuadrados = ProblemaGenetico([0,1],binario_a_decimal,fun_mutar, fun_cruzar, fun_fitness_cuad,10)

Una vez definida la instancia cuadrados que representa el problema genético, probar alguna de las funciones definidas en la clase anterior, para esta instancia concreta. Por ejemplo:

In [6]:
cuadrados.decodifica([1,0,0,0,1,1,0,0,1,0,1])
# Salida esperada: 1329

1329

In [7]:
cuadrados.fitness([1,0,0,0,1,1,0,0,1,0,1])
# Salida esperada: 1766241

1766241

In [17]:
cuadrados.muta([1,0,0,0,1,1,0,0,1,0,1],0.1)
# Posible salida: [1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

[1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

In [21]:
cuadrados.muta([1,0,0,0,1,1,0,0,1,0,1],0.1)
# Posible salida: [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]

[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]

In [22]:
cuadrados.cruza([1,0,0,0,1,1,0,0,1,0,1],[0,1,1,0,1,0,0,1,1,1])
# Posible salida: [[1, 0, 0, 0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]]

[[1, 0, 0, 0, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]]

### Ejercicio 3

   - Definir una función poblacion_inicial(problema_genetico,tamaño), para definir una población inicial de un tamaño dado, para una instancia dada de la clase anterior ProblemaGenetico

sugerencia: usar random.choice

   - Definir una función de cruce que recibe una instancia de Problema_Genetico y una población de padres (supondremos que hay un número par de padres), obtiene la población resultante de cruzarlos de dos en dos (en el orden en que aparecen)

cruza_padres(problema_genetico,padres)

   - Definir la función de mutación que recibe una instancia de Problema_Genetico, una población y una probabilidad de mutación, obtiene la población resultante de aplicar operaciones de mutación a cada individuo llamando a la función muta definida para el problema genético.
muta_individuos(problema_genetico, poblacion, prob)

In [126]:
def poblacion_inicial(problema_genetico, size):
    l = list()
    for i in range(size):
        indv = list()
        for j in range(problema_genetico.longitud_individuos):
            num = random.choice([0,1])
            indv.append(num)
        l.append(indv)
        
    return l

In [36]:
poblacion_inicial(cuadrados,10)

[[1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 1, 0, 1, 0, 0, 1, 0, 1],
 [1, 0, 1, 1, 1, 0, 0, 1, 1, 1],
 [0, 0, 1, 0, 0, 1, 0, 1, 1, 1],
 [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 0, 0, 0, 1, 1, 1, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 0, 0],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 1]]

In [127]:
def cruza_padres(problema_genetico,padres):
    l = list()
    for i in range(len(padres)//2):
        res_cruce = problema_genetico.cruza(padres[2*i],padres[2*i+1])
        l.append(res_cruce[0])
        l.append(res_cruce[1])
    return l

In [46]:
p1 = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
      [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
      [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
      [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
      [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
      [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

cruza_padres(cuadrados,p1)
# Posible salida
# [[1, 1, 0, 1, 0, 0, 1, 0, 1, 1],
#  [0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
#  [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
#  [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
#  [0, 1, 1, 1, 1, 0, 1, 1, 0, 1],
#  [1, 0, 1, 0, 0, 0, 0, 0, 0, 0]]

[[1, 1, 0, 1, 0, 0, 1, 0, 1, 1],
 [0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 1, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 0, 1, 1, 1, 0],
 [0, 1, 1, 0, 0, 0, 1, 1, 0, 1],
 [1, 0, 1, 1, 1, 0, 0, 0, 0, 0]]

In [128]:
def muta_individuos(problema_genetico, poblacion, prob):
    # hay que llamar a  problema_genetico.muta(x,prob) para todos los individuos de la poblacion.
    l = list()
    for i in range(len(poblacion)):
        l.append(problema_genetico.muta(poblacion[i],prob))
        
    return l

In [48]:
muta_individuos(cuadrados,p1,0.5)
# Posible salida:
#  [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
#   [0, 1, 0, 1, 0, 0, 1, 0, 0, 1],
#   [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
#   [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
#   [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#   [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

[[1, 1, 0, 1, 0, 1, 1, 0, 0, 1],
 [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
 [0, 1, 1, 0, 0, 0, 1, 1, 1, 0],
 [1, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 1, 0, 1, 1, 0, 0]]

In [49]:
p1 = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
      [0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
      [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
      [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
      [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
      [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

In [50]:
muta_individuos(cuadrados,p1,0.5)

[[1, 1, 0, 1, 0, 1, 0, 0, 0, 1],
 [0, 1, 1, 1, 0, 0, 1, 0, 1, 1],
 [0, 0, 1, 0, 0, 0, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 0, 1, 1, 1, 0],
 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 1, 0, 1, 1, 0, 1]]

### Ejercicio 4

Se pide definir una función de selección mediante torneo de n individuos de una población.  
La función recibe como entrada:
 - una instancia de la clase ProblemaGenetico
 - una población
 - el número n de individuos que vamos a seleccionar
 - el número k de participantes en el torneo
 - un valor opt que puede ser o la función max o la función min (dependiendo de si el problema es de maximización o de minimización, resp.).

seleccion\_por\_torneo(problema_genetico,poblacion,n,k,opt) 

INDICACIÓN: Usar random.sample para seleccionar k elementos de una secuencia. 
Por ejemplo, random.sample(population=[2,5,7,8,9], k=3) devuelve [7,5,8]. 

In [129]:
def seleccion_por_torneo(problema_genetico, poblacion, n, k, opt):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes
        y opt la función max o min."""
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion,k)
        seleccionado = opt(participantes, key=problema_genetico.fitness)
        opt(poblacion, key=problema_genetico.fitness)
        seleccionados.append(seleccionado)
        # poblacion.remove(seleccionado)
    return seleccionados  

In [18]:
#Ejemplo
seleccion_por_torneo(cuadrados, poblacion_inicial(cuadrados,8),3,6,max)
# Posible salida: [[1, 1, 1, 1, 1, 0, 0, 0, 1, 1], [1, 0, 0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 1, 0, 1, 1, 1, 0, 1]]


[[0, 0, 0, 1, 1, 1, 0, 0, 1, 1],
 [0, 0, 0, 1, 1, 1, 0, 0, 1, 1],
 [0, 0, 0, 1, 1, 1, 0, 0, 1, 1]]

In [20]:
seleccion_por_torneo(cuadrados, poblacion_inicial(cuadrados,8),3,6,min)
# [[0, 0, 1, 1, 0, 1, 1, 0, 0, 0], [1, 0, 1, 0, 1, 1, 1, 0, 0, 0], [1, 1, 0, 1, 0, 0, 1, 0, 1, 0]]

[[0, 0, 1, 1, 0, 1, 0, 1, 0, 0],
 [0, 0, 1, 1, 0, 1, 0, 1, 0, 0],
 [1, 1, 0, 1, 0, 0, 0, 0, 0, 0]]

In [121]:
# La siguiente función implementa una posibilidad para el algoritmo genético completo: 
# inicializa t = 0 
# Generar y evaluar la Población P(t)
# Mientras no hemos llegado al número de generaciones fijado:  t < nGen
#    P1 = Selección por torneo de (1-size)·p individuos de P(t)
#    P2 = Selección por torneo de (size·p) individuos de P(t)
#    Aplicar cruce en la población P2
#    P4 = Union de P1 y P3
#    P(t+1) := Aplicar mutación P4 
#    Evalua la población P(t+1) 
#    t:= t+1
        
# Sus argumentos son:
# problema_genetico: una instancia de la clase ProblemaGenetico con la representación adecuada del problema de optimización 
# que se quiere resolver.
# k: número de participantes en los torneos de selección.
# opt: max ó min, dependiendo si el problema es de maximización o de minimización. 
# nGen: número de generaciones (que se usa como condición de terminación)
# size: número de individuos en cada generación
# prop_cruce: proporción del total de la población que serán padres. 
# prob_mutación: probabilidad de realizar una mutación de un gen.

def algoritmo_genetico(problema_genetico,k,opt,ngen,size,prop_cruces,prob_mutar):
    poblacion= poblacion_inicial(problema_genetico,size)
    n_padres=round(size*prop_cruces)
    n_padres= int (n_padres if n_padres%2==0 else n_padres-1)
    n_directos = size-n_padres
    for _ in range(ngen):
        poblacion= nueva_generacion(problema_genetico,k,opt,poblacion,n_padres, n_directos,prob_mutar)

    mejor_cr= opt(poblacion, key=problema_genetico.fitness)
    mejor=problema_genetico.decodifica(mejor_cr)
    return (mejor,problema_genetico.fitness(mejor_cr)) 


Necesitarás definir la función auxiliar nueva_generacion(problema_genetico,poblacion,n_padres,n_directos,prob_mutar) que dada una población calcula la siguiente generación.

In [122]:
#Definir la función nueva_generacion
def nueva_generacion(problema_genetico, k,opt, poblacion, n_padres, n_directos, prob_mutar):
    padres2 = seleccion_por_torneo(problema_genetico, poblacion, n_directos, k,opt) 
    padres1 = seleccion_por_torneo(problema_genetico, poblacion, n_padres , k, opt)
    cruces =  cruza_padres(problema_genetico,padres1)
    generacion = padres2+cruces
    resultado_mutaciones = muta_individuos(problema_genetico, generacion, prob_mutar)
    return resultado_mutaciones

### Ejercicio 5.  Ejecutar el algoritmo genético anterior, para resolver el problema anterior (tanto en minimización como en maximización).  

Hacer una valoración de resultados y comentarios sobre el comportamiento del algoritmmo. 
En la resolución del problema hay que tener en cuenta que el algoritmo genético devuelve un par con el mejor fenotipo encontrado y su valoración.

In [32]:
algoritmo_genetico(cuadrados,3,min,20,10,0.7,0.1)
# Salida esperada: (0, 0)

(0, 0)

In [28]:
algoritmo_genetico(cuadrados,3,max,20,10,0.7,0.1)
# Salida esperada: (1023, 1046529)

(1023, 1046529)

#### Número de generaciones

Vamos a jugar primero con el número de generaciones, a ver cómo afecta a la solución obtenida.

In [15]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,3,min,5,10,0.7,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(1, 1), (8, 64), (2, 4), (0, 0), (35, 1225), (8, 64), (224, 50176), (2, 4), (220, 48400), (0, 0)]


In [16]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,3,min,10,10,0.7,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(32, 1024), (50, 2500), (2, 4), (225, 50625), (2, 4), (6, 36), (65, 4225), (0, 0), (8, 64), (2, 4)]


In [17]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,3,min,20,10,0.7,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(0, 0), (0, 0), (0, 0), (2, 4), (0, 0), (0, 0), (0, 0), (32, 1024), (0, 0), (0, 0)]


In [18]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,3,min,50,10,0.7,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]


In [19]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,3,max,5,10,0.7,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(925, 855625), (1020, 1040400), (1022, 1044484), (979, 958441), (1021, 1042441), (1012, 1024144), (831, 690561), (1015, 1030225), (1010, 1020100), (765, 585225)]


In [20]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,3,max,10,10,0.7,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(1023, 1046529), (1009, 1018081), (1022, 1044484), (1023, 1046529), (891, 793881), (959, 919681), (862, 743044), (1022, 1044484), (975, 950625), (1015, 1030225)]


In [21]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,3,max,20,10,0.7,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(1023, 1046529), (1023, 1046529), (1021, 1042441), (1022, 1044484), (1023, 1046529), (1007, 1014049), (1023, 1046529), (975, 950625), (1023, 1046529), (1023, 1046529)]


In [22]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,3,max,50,10,0.7,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(1023, 1046529), (1023, 1046529), (1023, 1046529), (1023, 1046529), (1023, 1046529), (1023, 1046529), (1023, 1046529), (1023, 1046529), (1023, 1046529), (1023, 1046529)]


Como se puede apreciar tanto en minimización como en maximización, cuanto mayor sea el número de generaciones, mejor solución obtenemos, llegando a la óptima. Con un número pequeño de generaciones hay mayor disparidad de soluciones entre cada iteración que llamamos al algoritmo, y según incrementamos ese número más cercanas son las soluciones entre sí y a la solución óptima.

#### Presión de selección

In [60]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,2,min,20,20,0.7,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(4, 16), (8, 64), (32, 1024), (0, 0), (0, 0), (0, 0), (1, 1), (1, 1), (0, 0), (2, 4)]


In [69]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,10,min,20,20,0.7,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(0, 0), (6, 36), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (64, 4096), (0, 0)]


In [63]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,15,min,20,20,0.7,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]


Vemos como cuanto mayor sea la presión de selección, mejores resultados obtenemos, ya que los peores individuos tienen menos posibilidades.

#### Proporción de cruce

In [86]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,3,min,20,10,0.1,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(810, 656100), (789, 622521), (820, 672400), (16, 256), (412, 169744), (417, 173889), (940, 883600), (353, 124609), (730, 532900), (975, 950625)]


In [85]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,3,min,20,10,0.7,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(0, 0), (16, 256), (0, 0), (2, 4), (0, 0), (68, 4624), (1, 1), (12, 144), (0, 0), (1, 1)]


Se puede apreciar que cuanto menor sea la proporción de cruce, peores resultados obtenemos. Esto se debe a que con valores bajos de proporción de cruce, es mayor la cantidad de individuos que pasan directamente a torneo y apenas hay variación en cada generación, por lo que depende mucho de la población inicial que tengamos, que siendo aleatoria es más probable que no nos acerque al óptimo, como se puede apreciar claramente en el ejemplo.

#### Probabilidad de mutación

In [87]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,3,min,20,10,0.7,0.01)
    soluciones.append(solucion)
    
print(soluciones)

[(70, 4900), (130, 16900), (17, 289), (8, 64), (70, 4900), (72, 5184), (34, 1156), (363, 131769), (44, 1936), (194, 37636)]


In [88]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,3,min,20,10,0.7,0.1)
    soluciones.append(solucion)
    
print(soluciones)

[(36, 1296), (4, 16), (32, 1024), (2, 4), (128, 16384), (0, 0), (0, 0), (0, 0), (0, 0), (4, 16)]


In [91]:
soluciones = list()
for i in range(10):
    solucion = algoritmo_genetico(cuadrados,3,min,20,10,0.7,0.3)
    soluciones.append(solucion)
    
print(soluciones)

[(0, 0), (0, 0), (0, 0), (0, 0), (1, 1), (0, 0), (0, 0), (0, 0), (2, 4), (0, 0)]


De nuevo, vemos que introducir la posibilidad de alejarnos de la población inicial, en este caso por medio de mutaciones, vemos que mejora la solución obtenida. 

##  El problema de la mochila 

Se plantea el típico problema de la mochila en el que dados n objetos de pesos conocidos pi y valor vi (i=1,...,n) hay que elegir cuáles se meten en una mochila que soporta un peso P máximo. La selección debe hacerse de forma que se máximice el valor de los objetos introducidos sin superar el peso máximo.

### Ejercicio 6
Se pide definir la representación del problema de la mochila usando genes [0,1] y longitud de los individuos n.

Los valores 1 ó 0 representan, respectivamente, si el objeto se introduce o no en la mochila Tomados de izquerda a derecha, a partir del primero que no cabe, se consideran  todos fuera de la mochila,independientemente del gen en su posición. De esta manera, todos los individuos representan candidatos válidos.

El numero de objetos n determina la longitud de los individuos de la población.
En primer lugar es necesario definir una función de decodificación de la mochila que recibe como entrada:
* un cromosoma (en este caso, una lista de 0s y 1s, de longitud igual a n_objetos) 
* n: número total de objetos de la mochila
* pesos: una lista con los pesos de los objetos
* capacidad: peso máximo de la mochila.
La función decodifica recibe (cromosoma, n, pesos, capacidad) y devuelve una lista de 0s y 1s que indique qué objetos están en la mochila y cuáles no (el objeto i está en la mochila si y sólo si en la posición i-ésima de la lista hay un 1). Esta lista se obtendrá a partir del cromosoma, pero teniendo en cuenta que a partir del primer objeto que no quepa, éste y los siguientes se consideran fuera de la mochila, independientemente del valor que haya en su correspondiente posición de cromosoma. 

In [112]:
def decodifica_mochila(cromosoma, n, pesos, capacidad):
    peso_en_mochila = 0
    l = []
    for i in range(n):
        if cromosoma[i] == 1 and peso_en_mochila + pesos[i] <= capacidad:
            l.append(1)
            peso_en_mochila += pesos[i]
        elif cromosoma[i]== 0 or peso_en_mochila + pesos[i] > capacidad:
            l.append(0)
    return l 

In [113]:
decodifica_mochila([1,1,1,1,1], 5, [2,3,4,5,1], 5)

[1, 1, 0, 0, 0]

Para definir la función de evaluación (fitness) necesitamos calcular el valor total de los objetos que están dentro de la mochila que representa el cromosoma según la codificación utilizada en la función anterior. 

Se pide la función fitness (cromosoma, n_objetos, pesos, capacidad, valores) donde los parámetros son los mismos que en la función anterior, y valores es la lista de los valores de cada objeto

fitness(cromosoma, n_objetos, pesos, capacidad, valores)

Ejemplo de uso:
   fitness([1,1,1,1], 4, [2,3,4,5], 4, [7,1,4,5])
   7

In [114]:
def fitness_mochila(cromosoma, n_objetos, pesos, capacidad, valores):
    dec = decodifica(cromosoma, n_objetos, pesos, capacidad)
    valor = 0
    for i in range(n_objetos):
        valor += dec[i]*valores[i]
        
    return valor

In [115]:
fitness_mochila([1,1,1,1], 4, [2,3,4,5], 4, [7,1,4,5])

7

Damos tres instancias concretas del problema de la mochila. Damos también sus soluciones optimas, para que se puedan comparar con los resultados obtenidos por el algoritmo genético:

In [116]:
# Problema de la mochila 1:
# 10 objetos, peso máximo 165
pesos1 = [23,31,29,44,53,38,63,85,89,82]
valores1 = [92,57,49,68,60,43,67,84,87,72]

# Solución óptima= [1,1,1,1,0,1,0,0,0,0], con valor 309

In [117]:
# Problema de la mochila 2:
# 15 objetos, peso máximo 750

pesos2 = [70,73,77,80,82,87,90,94,98,106,110,113,115,118,120]
valores2 = [135,139,149,150,156,163,173,184,192,201,210,214,221,229,240]

# Solución óptima= [1,0,1,0,1,0,1,1,1,0,0,0,0,1,1] con valor 1458

In [118]:
# Problema de la mochila 3:
# 24 objetos, peso máximo 6404180
pesos3 = [382745,799601,909247,729069,467902, 44328,
       34610,698150,823460,903959,853665,551830,610856,
       670702,488960,951111,323046,446298,931161, 31385,496951,264724,224916,169684]
valores3 = [825594,1677009,1676628,1523970, 943972,  97426,
       69666,1296457,1679693,1902996,
       1844992,1049289,1252836,1319836, 953277,2067538, 675367,
       853655,1826027, 65731, 901489, 577243, 466257, 369261]

# Solución óptima= [1,1,0,1,1,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,1] con valoración 13549094

### Ejercicio 7

Definir variables m1g, m2g y m3g, referenciando a instancias de Problema_Genetico que correspondan, respectivamente, a los problemas de la mochila anteriores. Resuelve los problemas y comentar los resultados obtenidos en cuanto a eficiencia y calidad de los resultados obtenidos.

Algunas de las salidas posibles variando los parámetros.

In [ ]:
# >>> algoritmo_genetico_t(m1g,3,max,100,50,0.8,0.05)
# ([1, 1, 1, 1, 0, 1, 0, 0, 0, 0], 309)

# >>> algoritmo_genetico_t(m2g,3,max,100,50,0.8,0.05)
# ([1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0], 1444)
# >>> algoritmo_genetico_t(m2g,3,max,200,100,0.8,0.05)
# ([0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0], 1439)
# >>> algoritmo_genetico_t(m2g,3,max,200,100,0.8,0.05)
# ([1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1], 1458)

# >>> algoritmo_genetico_t(m3g,5,max,400,200,0.75,0.1)
# ([1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], 13518963)
# >>> algoritmo_genetico_t(m3g,4,max,600,200,0.75,0.1)
# ([1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0], 13524340)
# >>> algoritmo_genetico_t(m3g,4,max,1000,200,0.75,0.1)
# ([1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], 13449995)
# >>> algoritmo_genetico_t(m3g,3,max,1000,100,0.75,0.1)
# ([1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], 13412953)
# >>> algoritmo_genetico_t(m3g,3,max,2000,100,0.75,0.1)
# ([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 13366296)
# >>> algoritmo_genetico_t(m3g,6,max,2000,100,0.75,0.1)
# ([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1], 13549094)

In [119]:

def fitness_mochila_1(cromosoma):
    v = fitness_mochila(cromosoma, 10, pesos1, 165, valores1)
    return v
def decodifica_mochila_1(cromosoma):
    v = decodifica_mochila(cromosoma, 10, pesos1, 165)
    return v
m1g = ProblemaGenetico([0,1], decodifica_mochila_1, fun_mutar, fun_cruzar, fitness_mochila_1,10)

def fitness_mochila_2(cromosoma):
    v = fitness_mochila(cromosoma, 15, pesos2, 750, valores2)
    return v
def decodifica_mochila_2(cromosoma):
    v = decodifica_mochila(cromosoma, 14, pesos2, 750)
    return v
m2g = ProblemaGenetico([0,1], decodifica_mochila_2, fun_mutar, fun_cruzar, fitness_mochila_2,15)

def fitness_mochila_3(cromosoma):
    v = fitness_mochila(cromosoma, 24, pesos3,6404180 , valores3)
    return v
def decodifica_mochila_3(cromosoma):
    v = decodifica_mochila(cromosoma, 24, pesos3, 6404180)
    return v
m3g = ProblemaGenetico([0,1], decodifica_mochila_3, fun_mutar, fun_cruzar, fitness_mochila_3,24)


Para cada mochila, vamos a ejecutar el algoritmo una serie de veces y ver el valor promedio que obtenemos en esas iteraciones, de modo que tengamos una visión clara de en qué condiciones nos acercamos en general más a la solución óptima.

### m1g

In [137]:
sol = 0
for i in range(100):
    sol += algoritmo_genetico(m1g,3,max,100,50,0.8,0.05)[1]

print(sol/100)

308.11


In [138]:
#Disminuimos el número de generaciones a 50
sol = 0
for i in range(100):
    sol += algoritmo_genetico(m1g,3,max,50,50,0.8,0.05)[1]

print(sol/100)

307.5


In [139]:
#Disminuimos el número de generaciones a 20
sol = 0
for i in range(100):
    sol += algoritmo_genetico(m1g,3,max,20,50,0.8,0.05)[1]

print(sol/100)

307.5


Como vemos, modificar el número de generaciones no afecta considerablemente el resultado, ya que en los tres obtenemos valores buenos. Hay que tener en cuenta que estos resultados dependen mucho del caso particular, por lo que la conclusión que debemos sacar es simplemente que nos sigue dando buenas soluciones. Obviamente si nos vamos a un número muy pequeño de generaciones el resultado empeorará.

In [143]:
#Presión de selección 1
sol = 0
for i in range(100):
    sol += algoritmo_genetico(m1g,1,max,100,50,0.8,0.05)[1]

print(sol/100)

270.14


In [144]:
#Presión de selección 10
sol = 0
for i in range(100):
    sol += algoritmo_genetico(m1g,1,max,100,50,0.8,0.05)[1]

print(sol/100)

271.52


Reducir la presión de selección nos da peores resultados, como ya hemos comentado anteriormente, pero como también podemos ver que incrementarlo demasiado nos empeora el resultado. Esto puede deberse a que, a pesar de quedarse con individuos muy buenos, puede dar lugar a poca variabilidad genética.

In [147]:
#Proporción de cruce 0.4
sol = 0
for i in range(100):
    sol += algoritmo_genetico(m1g,3,max,100,50,0.4,0.05)[1]

print(sol/100)

307.75


In [148]:
#Proporción de cruce 0.1
sol = 0
for i in range(100):
    sol += algoritmo_genetico(m1g,3,max,100,50,0.1,0.05)[1]

print(sol/100)

308.5


En el caso de modificar la proporción de cruce no se ha producido ningún cambio considerable.

In [149]:
#Probabilidad de mutación 0.1
sol = 0
for i in range(100):
    sol += algoritmo_genetico(m1g,3,max,100,50,0.8,0.1)[1]

print(sol/100)

307.75


In [150]:
#Probabilidad de mutación 0.5
sol = 0
for i in range(100):
    sol += algoritmo_genetico(m1g,3,max,100,50,0.8,0.5)[1]

print(sol/100)

309.0


Aumentar la probabilidad de mutación nos ha dado mejores resulados, especialmente el caso 0.5 en el que las 100 llamadas al algoritmo genético han dado el valor óptimo.

### m2g
A partir de aquí vamos a reducir el número de llamadas al algoritmo con el fin de reducir el tiempo de espera a que cada llamada termine.

In [151]:
sol = 0
for i in range(50):
    sol += algoritmo_genetico(m2g,3,max,100,50,0.8,0.05)[1]

print(sol/50)

1442.1


In [152]:
sol = 0
for i in range(50):
    sol += algoritmo_genetico(m2g,3,max,200,100,0.8,0.05)[1]

print(sol/50)

1444.24


In [153]:
sol = 0
for i in range(50): 
    sol += algoritmo_genetico(m2g,3,max,200,100,0.8,0.2)[1]
    
print(sol/50)

1447.5


In [154]:
sol = 0
for i in range(50): 
    sol += algoritmo_genetico(m2g,3,max,200,100,0.8,0.4)[1]
    
print(sol/50)

1450.64


De nuevo, al igual que en la configuración de problema anterior, se puede apreciar cómo apenas varía el promedio si modificamos el número de generaciones o el tamaño de la población. Lo que sí mejora el promedio es el incremento de la probabilidad de mutación.

### m3g

In [159]:
algoritmo_genetico(m3g,5,max,400,200,0.75,0.1)

([1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1],
 13461188)

In [160]:
algoritmo_genetico(m3g,4,max,600,200,0.75,0.1)

([1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 13515028)

In [161]:
algoritmo_genetico(m3g,4,max,1000,200,0.75,0.1)

([1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
 13518963)

In [162]:
algoritmo_genetico(m3g,3,max,1000,100,0.75,0.1)

([1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 13515028)

In [163]:
algoritmo_genetico(m3g,3,max,2000,100,0.75,0.1)

([1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0],
 13524340)

In [164]:
algoritmo_genetico(m3g,6,max,2000,100,0.75,0.1)

([1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1],
 13496748)

### Ejercicio 8
Resolver mediante una configuración de un algoritmo genético el problema del cuadrado mágico que consiste en colocar en un cuadrado n × n los números naturales de 1 a n^2, 
de tal manera que las filas, las columnas y las diagonales principales sumen los mismo. 
Ejemplo: una solucion para n= 3
    
    4 3 8
    9 5 1
    2 7 6
    
- Dimensionn del cuadrado: N
- Suma común: SUMA=(N·(N^2 + 1))/2
    
    Comenta el resultado y el rendimiento del algoritmo para distintos parámetros.
    

    
    

### Representación del problema
Del mismo modo que en el problema del 8-puzzle, en este problema cada cromosoma va a ser una lista de tamaño n·n, formada por números del 1 a n·n sin repetir ninguno. Utilizaremos la clase ProblemaGenético ya definida más arriba.

### Definición de funciones de cruce, mutación y fitness

In [102]:
import math

def cruce_de_orden(cromosoma1, cromosoma2, c1, c2):
    #Dados dos cromosomas, realiza el cruce de orden
    l = len(cromosoma1)
    cadena_central = cromosoma1[c1:c2]
    cadena_aux = list()
    j = c2
    for i in range(l):
        if j==l:
            j = 0
        if cromosoma2[j] not in cadena_central:
            cadena_aux.append(cromosoma2[j])
        j += 1
    
    l_aux = len(cadena_aux)
    return cadena_aux[l-c2:l_aux] + cadena_central + cadena_aux[0:l-c2]

def fun_cruzar_cuadrado(cromosoma1, cromosoma2):
    """Cruce de orden"""
    #Elegimos sección aleatoria de cromosoma1 desde c1 a c2-1, recorremos cromosoma2 desde c2 llenando primero la 
    #cadena derecha, y luego la izquierda de los elementos de cromosoma2 que no están en la sección escogida de cromosoma1
    l = len(cromosoma1)
    c1 = random.randint(0,l-1)
    c2 = random.randint(0,l-1)
    if c2 < c1:
        temp = c2
        c2 = c1
        c1 = temp
    c2 += 1
    cruce1 = cruce_de_orden(cromosoma1, cromosoma2, c1, c2)
    cruce2 = cruce_de_orden(cromosoma2, cromosoma1, c1, c2)
    
    return [cruce1,cruce2]

def fun_mutar_cuadrado(cromosoma,prob):
    """Elige una sección al azar del cromosoma y la revuelve"""
    l = len(cromosoma)
    c1 = random.randint(0,l-1)
    c2 = random.randint(0,l-1)
    if c2 < c1:
        temp = c2
        c2 = c1
        c1 = temp
    c2 += 1
    if prob > random.uniform(0,1):
        cromosoma[c1:c2] =  random.sample(cromosoma[c1:c2],c2-c1)
        
    return cromosoma

def fun_fitness_cuadrado(cromosoma):
    """Calculamos la suma de los elementos de cada fila/columna/diagonal, y calculamos el módulo entre ese valor y 
    el valor suma buscado. El valor fitness será la suma de todas esas diferencias, valiendo 0 si todos coinciden
    en suma, y más valor cuanto más alejado de la solución se encuentre el tablero. Así el problema consiste en minimizar
    el valor de la función de fitness."""
    
    N = int(math.sqrt(len(cromosoma)))
    valor_magico = (N*(N**2 + 1))//2
    fitness_val = 0
    for i in range(N):
        fitness_fila = valor_magico
        fitness_columna = valor_magico
        for j in range(N):
            fitness_fila -= cromosoma[N*i+j]
            fitness_columna -= cromosoma[N*j+i]
        fitness_val += abs(fitness_fila)
        fitness_val += abs(fitness_columna)
        
    fitness_diag1 = valor_magico
    fitness_diag2 = valor_magico        
    for i in range(N):
        fitness_diag1 -= cromosoma[N*i+i]
        fitness_diag2 -= cromosoma[N*(i+1)-(i+1)]
    fitness_val += abs(fitness_diag1)
    fitness_val += abs(fitness_diag2)
    
    return fitness_val

In [99]:
fun_fitness_cuadrado([4,1,8,9,5,3,2,7,6])

8

In [100]:
fun_fitness_cuadrado([4,3,8,9,5,1,2,7,6])

0

In [67]:
fun_mutar_cuadrado([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], 0.99)

[1, 2, 3, 4, 12, 8, 9, 5, 7, 6, 11, 14, 13, 10, 15]

In [109]:
fun_cruzar_cuadrado([1,2,3,4,5,6,7,8,9], [1,2,4,3,8,7,6,5,9])

[[1, 2, 3, 4, 5, 6, 9, 8, 7], [1, 2, 4, 3, 8, 7, 9, 5, 6]]

### Definimos la función de población inicial, decodificación y algoritmo genético

In [130]:
def poblacion_inicial_perm(problema_genetico, size):
    """Dadas las dimensiones del cuadrado, nos da una permutación aleatoria de los valores"""
    l = list()
    for i in range(size):
        N = problema_genetico.longitud_individuos**2
        indv = random.sample(list(range(1,N+1)),N)
        l.append(indv)
        
    return l

In [131]:
def decodifica_cuadrado(cromosoma):
    """Con nuestra representación no hay que hacer nada"""
    return cromosoma

In [132]:
def algoritmo_genetico_cuadrado(problema_genetico,k,opt,ngen,size,prop_cruces,prob_mutar):
    poblacion= poblacion_inicial_perm(problema_genetico,size)
    n_padres=round(size*prop_cruces)
    n_padres= int (n_padres if n_padres%2==0 else n_padres-1)
    n_directos = size-n_padres
    for _ in range(ngen):
        poblacion= nueva_generacion(problema_genetico,k,opt,poblacion,n_padres, n_directos,prob_mutar)

    mejor_cr= opt(poblacion, key=problema_genetico.fitness)
    mejor=problema_genetico.decodifica(mejor_cr)
    return (mejor,problema_genetico.fitness(mejor_cr)) 

### Vamos a ver 2 ejemplos de este problema

In [133]:
cuadrado3 = ProblemaGenetico(list(range(1,3**2)), decodifica_cuadrado, fun_mutar_cuadrado, fun_cruzar_cuadrado, fun_fitness_cuadrado,3)
cuadrado10 = ProblemaGenetico(list(range(1,10**2)), decodifica_cuadrado, fun_mutar_cuadrado, fun_cruzar_cuadrado, fun_fitness_cuadrado,10)

In [143]:
algoritmo_genetico_cuadrado(cuadrado3,3,min,100,50,0.8,0.05)

([4, 3, 8, 9, 5, 1, 2, 7, 6], 0)

In [150]:
soluciones = list()
suma = 0
for i in range(50):
    solucion = algoritmo_genetico_cuadrado(cuadrado3,3,min,100,50,0.8,0.05)[1]
    suma += solucion
    soluciones.append(solucion)
print(soluciones)
print("Promedio: " + str(suma/50))

[9, 6, 9, 2, 6, 5, 5, 6, 7, 5, 0, 0, 6, 6, 8, 9, 0, 8, 3, 0, 6, 0, 2, 6, 8, 0, 6, 6, 8, 6, 3, 2, 3, 7, 7, 10, 6, 9, 14, 6, 7, 7, 5, 0, 0, 8, 8, 7, 5, 6]
Promedio: 5.36


In [151]:
soluciones = list()
suma = 0
for i in range(50):
    solucion = algoritmo_genetico_cuadrado(cuadrado3,3,min,200,50,0.8,0.05)[1]
    suma += solucion
    soluciones.append(solucion)
print(soluciones)
print("Promedio: " + str(suma/50))

[5, 6, 5, 0, 7, 5, 3, 7, 7, 7, 5, 7, 5, 9, 7, 6, 6, 5, 10, 7, 0, 6, 5, 11, 3, 8, 7, 7, 3, 5, 6, 6, 5, 11, 6, 5, 9, 9, 2, 0, 9, 7, 6, 0, 5, 6, 2, 5, 6, 6]
Promedio: 5.7


In [153]:
soluciones = list()
suma = 0
for i in range(50):
    solucio = algoritmo_genetico_cuadrado(cuadrado3,10,min,200,50,0.8,0.05)[1]
    suma += solucion
    soluciones.append(solucion)
print(soluciones)
print("Promedio: " + str(suma/50))

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
Promedio: 6.0


In [154]:
soluciones = list()
suma = 0
for i in range(50):
    solucion = algoritmo_genetico_cuadrado(cuadrado3,10,min,200,50,0.8,0.2)[1]
    suma += solucion
    soluciones.append(solucion)
print(soluciones)
print("Promedio: " + str(suma/50))

[3, 2, 0, 5, 3, 3, 3, 0, 8, 8, 6, 6, 7, 7, 0, 2, 5, 5, 6, 5, 2, 0, 5, 3, 0, 5, 3, 5, 6, 6, 5, 5, 2, 3, 5, 7, 0, 5, 3, 0, 0, 0, 7, 3, 5, 5, 6, 6, 0, 6]
Promedio: 3.84


In [155]:
soluciones = list()
suma = 0
for i in range(50):
    solucion = algoritmo_genetico_cuadrado(cuadrado3,10,min,200,50,0.8,0.5)[1]
    suma += solucion
    soluciones.append(solucion)
print(soluciones)
print("Promedio: " + str(suma/50))

[3, 3, 5, 0, 7, 6, 6, 2, 2, 6, 2, 0, 0, 2, 7, 5, 2, 3, 5, 0, 7, 6, 2, 3, 2, 0, 3, 5, 3, 3, 0, 3, 2, 5, 0, 5, 8, 8, 6, 0, 0, 5, 0, 6, 5, 0, 0, 3, 0, 0]
Promedio: 3.12


In [162]:
soluciones = list()
suma = 0
for i in range(50):
    solucion = algoritmo_genetico_cuadrado(cuadrado10,3,min,100,50,0.8,0.05)[1]
    suma += solucion
    soluciones.append(solucion)
print(soluciones)
print("Promedio: " + str(suma/50))

[572, 465, 536, 584, 641, 404, 535, 616, 623, 521, 516, 512, 549, 427, 568, 482, 584, 543, 507, 622, 622, 537, 670, 543, 603, 532, 620, 570, 428, 501, 497, 494, 472, 559, 446, 543, 612, 523, 473, 413, 425, 518, 651, 524, 567, 690, 565, 583, 578, 429]
Promedio: 539.9


In [161]:
soluciones = list()
suma = 0
for i in range(50):
    solucion = algoritmo_genetico_cuadrado(cuadrado10,3,min,200,100,0.8,0.05)[1]
    suma += solucion
    soluciones.append(solucion)
print(soluciones)
print("Promedio: " + str(suma/50))

[500, 302, 413, 405, 461, 454, 355, 504, 294, 466, 481, 319, 353, 377, 440, 307, 384, 527, 375, 345, 301, 367, 304, 270, 382, 495, 523, 391, 432, 306, 292, 402, 385, 418, 284, 351, 368, 418, 341, 382, 471, 327, 350, 445, 484, 365, 348, 613, 452, 437]
Promedio: 395.32


In [159]:
soluciones = list()
suma = 0
for i in range(50):
    solucio = algoritmo_genetico_cuadrado(cuadrado3,10,min,100,50,0.8,0.05)[1]
    suma += solucion
    soluciones.append(solucion)
print(soluciones)
print("Promedio: " + str(suma/50))

[418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418, 418]
Promedio: 418.0


In [160]:
soluciones = list()
suma = 0
for i in range(50):
    solucion = algoritmo_genetico_cuadrado(cuadrado10,3,min,100,50,0.8,0.2)[1]
    suma += solucion
    soluciones.append(solucion)
print(soluciones)
print("Promedio: " + str(suma/50))

[487, 482, 323, 458, 510, 470, 426, 524, 487, 551, 510, 537, 379, 471, 376, 671, 575, 511, 429, 450, 634, 453, 422, 471, 533, 469, 367, 436, 453, 500, 407, 515, 450, 321, 525, 571, 329, 334, 490, 594, 382, 379, 465, 438, 725, 438, 493, 509, 381, 593]
Promedio: 474.08


Se puede apreciar que a mayor dimensión del cuadrado, más se nos complica la solución, en el sentido de que el promedio se nos aleja bastante de lo buscado. Esto puede deberse a que las pruebas se han hecho con un número de generaciones y de población bajo, ya que de lo contrario el algoritmo tardaría demasiado. 

Además hay que tener en cuenta la naturaleza del problema, en el hecho de que que una fila/columna/diagonal no sea correcta afecta a todas las demás, dando lugar a que el valor de fitness sea elevado muy fácilmente. Por esto mismo, además cualquier mutación o cruce afectarán en gran medida este valor.

### Ejercicio 9
Se quiere organizar una **jornada de la informática** en la Facultad. 
El Vicedecano necesita organizar el horario y la asignación de profesores para n actividades temáticas, a_1, . . . , a_n.  Dispone de m profesores p_1, . . . , p_m con distintos niveles de cualificación. 
No todos los profesores tienen los conocimientos para encargarse de todas las actividades. 
El mismo profesor pi puede encargarse de distintas actividades (que se le remunerarán convenientemente) pero no podemos asignar al mismo profesor dos actividades que se impartan simultáneamente. 
Cada actividad puede durar un tiempo diferente dependiendo del profesor que la organice. Si un profesor p_i puede realizar la actividad a_j, entonces denotaremos por t_ij el tiempo que pi tardará en realizar la actividad a_j. 
Existen dependencias entre los contenidos de las actividades lo que obliga a incluir algunas restricciones temporales, es decir, algunas actividades hay que realizarlas cuando otras ya hayan terminado. Esto es, tenemos un conjunto de restricciones que indican que ai se debe realizar antes de a_j. Por ejemplo, El taller de iniciación a Scratch se debe realizar antes que el laboratorio de desarrollo de videojuegos con Scratch.   
Se pide dar una solución al problema de decidir, para cada actividad, qué profesor la realizará y en qué momento se debe ofertar la actividad para que, al ﬁnal de la jornada, se puedan realizar todas las actividades con cuidado de cumplir los prerrequisitos (restricciones temporales) y el conjunto de actividades se realice en el menor tiempo posible.

### Representación del problema

Para la representación del problema, lo resolveremos con un algoritmo genético, usando valores enteros en lugar de codificación 0-1. Esto nos simplifica los cáluclos en el problema, ya que no hay mucha diferencia tanto en el cruce como en las mutaciones el hacerlo con una codificación u otra. Por ejemplo, es muy similar cambiar un bit aleatorio de 0 a 1 que cambiar de forma aleatoria el profesor asignado a una actividad (también en ambos casos habría que comprobar que el nuevo profesor puede realizar la actividad). 

Podemos considerar que un cromosoma está dividido en dos partes, ambas de longitud igual al número de actividades. La primera representa para actividad el profesor que se encarga de ella. La segunda representa el orden en el que se deben realizar las actividades sin romper ninguna de las restricciones de dependencia. Así tenemos un cromosoma cuyo tamaño es el doble de la cantidad de actividades a repartir. De este modo haremos que todas las operaciones de cruce y mutación solo afecten a la primera mitad de la cadena, por lo que siempre darán lugar a un individuo válido (ya que el orden de las actividades seguirá siendo válido). 

Por ejemplo, para n=4 y m=3 podemos tener el cromosoma: [0,1,0,2,3,1,0,2]

    - La tarea 0 la realiza la persona 0.
    - La tarea 1 la realiza la persona 1.
    - La tarea 2 la realiza la persona 0.
    - La tarea 3 la realiza la persona 2.
    - Las tareas se realizan en orden 3 -> 1 -> 0 -> 2
                 

In [24]:
# Definimos una clase para los datos de la entrada.
# - n_actividades: número de actividades a realizar.
# - n_profesores: número de profesores que se van a repartir las actividades.
# - cualif_actividades: lista de listas de profesores que pueden realizar cada actividad.
# - tiempo_actividades: lista de listas en la que para cada profesor se indica el tiempo que tarda en realizar cada actividad.
#   (Por simplicidad, vamos a suponer que aunque un profesor no esté cualificado para realizar una actividad, tiene igualmente 
#   un tiempo asignado para ella).
# - dependencia_actividades: lista de listas en la que para cada actividad se indica las actividades que deben realizarse 
#   después de ella (como si fuera la lista de adyacencia de un grafo).

class DatosActividades():
    def __init__(self, n_actividades, n_profesores, cualif_actividades, tiempo_actividades, dependencia_actividades):
        self.n_actividades = n_actividades
        self.n_profesores = n_profesores
        self.cualif_actividades = cualif_actividades
        self.tiempo_actividades = tiempo_actividades
        self.dependencia_actividades = dependencia_actividades
    
class ProblemaGenetico(object):
        def __init__(self, fun_dec, fun_muta , fun_cruza, fun_fitness, datos_actividades):
            self.fun_dec = fun_dec
            self.fun_cruza = fun_cruza
            self.fun_muta = fun_muta
            self.fun_fitness = fun_fitness
            self.datos_actividades = datos_actividades
            """Constructor de la clase"""
                
        def decodifica(self, genotipo):
            """Devuelve el fenotipo a partir del genotipo"""
            fenotipo = self.fun_dec(genotipo)
            return fenotipo
        def muta(self, cromosoma,prob):
            """Devuelve el cromosoma mutado"""   
            mutante = self.fun_muta(cromosoma,prob,self.datos_actividades.cualif_actividades)
            return mutante
        def cruza(self, cromosoma1, cromosoma2):         
            """Devuelve el cruce de un par de cromosomas"""
            cruce = self.fun_cruza(cromosoma1,cromosoma2)
            return cruce 
        def fitness(self, cromosoma):    
            """Función de valoración"""
            valoracion = self.fun_fitness(cromosoma, self.datos_actividades.dependencia_actividades, self.datos_actividades.tiempo_actividades)
            return valoracion

In [25]:
import random
# Para obtener el orden en el que se deben realizar las actividades, hacemos un topological sort (Algoritmo de Kahn).
def topological_sort(dependencia_actividades):
    queue = list()
    in_degree = list()
    actividades_ordenadas = list()
    for i in range(len(dependencia_actividades)):
        in_degree.append(0)
    
    for i in range(len(dependencia_actividades)):
        for j in dependencia_actividades[i]:
            in_degree[j] += 1
            
    for i in range(len(in_degree)):
        if in_degree[i] == 0:
            queue.append(i)
    
    while queue:
        #Extraemos un elemento aleatorio de la cola para tener distintas soluciones en cada ejecución
        random_node = random.randint(0,len(queue)-1)
        a = queue.pop(random_node)
        actividades_ordenadas.append(a)
        for i in dependencia_actividades[a]:
            in_degree[i] -= 1
            if in_degree[i] == 0:
                queue.append(i)
    
    if len(actividades_ordenadas) != len(dependencia_actividades):
        print("Hay un ciclo en la dependencia de tareas")
    
    return actividades_ordenadas

In [26]:
sort_test = [[],[],[3],[1],[0,1],[2,0]]
for i in range(50):
    print(topological_sort(sort_test))

[5, 4, 2, 0, 3, 1]
[4, 5, 2, 0, 3, 1]
[5, 2, 4, 3, 1, 0]
[4, 5, 0, 2, 3, 1]
[5, 4, 2, 0, 3, 1]
[4, 5, 0, 2, 3, 1]
[5, 4, 0, 2, 3, 1]
[5, 4, 2, 0, 3, 1]
[5, 2, 4, 0, 3, 1]
[5, 4, 2, 0, 3, 1]
[4, 5, 2, 3, 1, 0]
[5, 2, 3, 4, 0, 1]
[5, 4, 0, 2, 3, 1]
[4, 5, 2, 3, 1, 0]
[5, 2, 3, 4, 0, 1]
[5, 4, 2, 0, 3, 1]
[4, 5, 2, 3, 0, 1]
[4, 5, 0, 2, 3, 1]
[5, 2, 3, 4, 1, 0]
[4, 5, 2, 0, 3, 1]
[5, 4, 0, 2, 3, 1]
[4, 5, 0, 2, 3, 1]
[4, 5, 0, 2, 3, 1]
[4, 5, 2, 3, 0, 1]
[5, 4, 0, 2, 3, 1]
[5, 2, 4, 0, 3, 1]
[5, 2, 4, 3, 0, 1]
[4, 5, 2, 3, 1, 0]
[5, 4, 0, 2, 3, 1]
[4, 5, 0, 2, 3, 1]
[5, 2, 3, 4, 1, 0]
[4, 5, 2, 0, 3, 1]
[5, 4, 0, 2, 3, 1]
[5, 2, 3, 4, 0, 1]
[5, 4, 2, 3, 0, 1]
[5, 4, 2, 3, 1, 0]
[4, 5, 0, 2, 3, 1]
[4, 5, 0, 2, 3, 1]
[5, 2, 3, 4, 0, 1]
[5, 4, 2, 0, 3, 1]
[4, 5, 0, 2, 3, 1]
[5, 2, 4, 0, 3, 1]
[4, 5, 2, 3, 0, 1]
[4, 5, 0, 2, 3, 1]
[5, 4, 2, 0, 3, 1]
[5, 2, 3, 4, 0, 1]
[5, 4, 2, 0, 3, 1]
[5, 2, 4, 0, 3, 1]
[5, 2, 3, 4, 0, 1]
[4, 5, 0, 2, 3, 1]


### Definición de función de cruce, mutación y fitness

Para el cruce, elegimos un punto de corte aleatorio y a partir de ahí intercambiamos las secciones entre los dos padres. Pueso que ya tenemos el orden de las tareas establecido, esto no afectará a ninguna de las restricciones de orden.

Para la mutación, escogemos de forma aleatoria una actividad. Después cambiamos la persona asignada por otra cualificada para esa tarea, elegida también de forma aleatoria y distinta de la que estaba asignada inicialmente.

La función de fitness es el tiempo en el que acaba de realizarse la última actividad dada la distribución y orden en el que se realizan. Inicialmente todas están en el instante 0, y el tiempo que tarda una actividad es el tiempo acumulado más su duración.

In [27]:
def fun_cruza_actividades(cromosoma1, cromosoma2):
    l = len(cromosoma1)
    ## Entre 1 y l-1 para que el corte no nos dé secciones vacías
    c = random.randint(1,l//2-1)
    cruce1 = cromosoma1[0:c] + cromosoma2[c:l//2] + cromosoma1[l//2:l]
    cruce2 = cromosoma2[0:c] + cromosoma1[c:l//2] + cromosoma2[l//2:l]
    
    return [cruce1,cruce2]

def muta_actividades(cromosoma, prob, cualif_actividades):
    """Elige una actividad al azar y cambia la persona asignada por otra aleatoria"""
    l = len(cromosoma)//2
    c = random.randint(0,l-1)
    if prob > random.uniform(0,1):
        #Elige una persona aleatoria de la lista de cualificados para esa actividad
        cromosoma[c] =  random.choice(cualif_actividades[c])
        
    return cromosoma
    
def fitness_actividades(cromosoma, dependencia_actividades, tiempo_actividades):
    tiempo_max = 0
    tiempos_actividades = list()
    l = len(cromosoma)//2
    for i in range(l):
        tiempos_actividades.append(0)
    dependencia_actividades_rev = [[] for i in range(l)]
    
    for i in range(l):
        for j in range(len(dependencia_actividades[i])):
            dependencia_actividades_rev[dependencia_actividades[i][j]].append(i)
            
    for i in range(l):
        cont = i - 1
        while cont >= 0 and cromosoma[i] != cromosoma[cont]:
            cont -= 1
        if cont >= 0:
            dependencia_actividades_rev[cromosoma[l+i]].append(cromosoma[l+cont])
        
    for i in range(l):
        actividad_actual = cromosoma[l+i]
        for j in range(len(dependencia_actividades_rev[actividad_actual])):
            tiempos_actividades[actividad_actual] = max(tiempos_actividades[actividad_actual], tiempos_actividades[dependencia_actividades_rev[actividad_actual][j]])
        tiempos_actividades[actividad_actual] += tiempo_actividades[cromosoma[i]][actividad_actual]
        tiempo_max = max(tiempo_max, tiempos_actividades[actividad_actual])
        
    return tiempo_max

### Definimos la función de población inicial, decodificación y algoritmo genético

In [34]:
def poblacion_inicial_actividades(problema_genetico, size):
    l = list()
    for i in range(size):
        n = problema_genetico.datos_actividades.n_actividades
        m = problema_genetico.datos_actividades.n_profesores
        random_profesores = list()
        for i in range(n):
            random_profesores.append(random.choice(problema_genetico.datos_actividades.cualif_actividades[i]))
        random_orden_actividades = topological_sort(problema_genetico.datos_actividades.dependencia_actividades)
        indv = random_profesores + random_orden_actividades
        l.append(indv)
        
    return l

def decodifica_actividades(cromosoma):
    """Con nuestra representación no hay que hacer nada"""
    return cromosoma

def cruza_padres(problema_genetico,padres):
    l = list()
    for i in range(len(padres)//2):
        res_cruce = problema_genetico.cruza(padres[2*i],padres[2*i+1])
        l.append(res_cruce[0])
        l.append(res_cruce[1])
    return l

def muta_individuos(problema_genetico, poblacion, prob):
    # hay que llamar a  problema_genetico.muta(x,prob) para todos los individuos de la poblacion.
    l = list()
    for i in range(len(poblacion)):
        l.append(problema_genetico.muta(poblacion[i],prob))
        
    return l

def seleccion_por_torneo(problema_genetico, poblacion, n, k, opt):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes
        y opt la función max o min."""
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion,k)
        seleccionado = opt(participantes, key=problema_genetico.fitness)
        opt(poblacion, key=problema_genetico.fitness)
        seleccionados.append(seleccionado)
        # poblacion.remove(seleccionado)
    return seleccionados

def nueva_generacion(problema_genetico, k,opt, poblacion, n_padres, n_directos, prob_mutar):
    padres2 = seleccion_por_torneo(problema_genetico, poblacion, n_directos, k,opt) 
    padres1 = seleccion_por_torneo(problema_genetico, poblacion, n_padres , k, opt)
    cruces =  cruza_padres(problema_genetico,padres1)
    generacion = padres2+cruces
    resultado_mutaciones = muta_individuos(problema_genetico, generacion, prob_mutar)
    return resultado_mutaciones

def algoritmo_genetico_actividades(problema_genetico,k,opt,ngen,size,prop_cruces,prob_mutar):
    poblacion= poblacion_inicial_actividades(problema_genetico,size)
    n_padres=round(size*prop_cruces)
    n_padres= int (n_padres if n_padres%2==0 else n_padres-1)
    n_directos = size-n_padres
    for _ in range(ngen):
        poblacion= nueva_generacion(problema_genetico,k,opt,poblacion,n_padres, n_directos,prob_mutar)

    mejor_cr= opt(poblacion, key=problema_genetico.fitness)
    mejor=problema_genetico.decodifica(mejor_cr)
    return (mejor,problema_genetico.fitness(mejor_cr)) 

In [35]:
import random

n = 8
m = 3

#Tiempo de actividades
tiempo_actividades = list()
for i in range(m):
    t_actividad = list()
    for j in range(n):
        t_actividad.append(random.randint(50,100))

    tiempo_actividades.append(t_actividad)

print("Tiempo realización actividades:")
print(tiempo_actividades)
    
#Personas que pueden realizar cada actividad
cualif_actividades = list()
for i in range(n):
    c_actividad = random.sample(list(range(m)),random.randint(1,m-1))
    cualif_actividades.append(c_actividad)

print("Cualificación para realización actividades:")    
print(cualif_actividades)

#Dependencia entre actividades
dependencia_actividades = list()
for i in range(n-1):
    d_actividad = random.sample(list(range(i+1,n-1)),random.randint(0,n-1-i-1))    
    dependencia_actividades.append(d_actividad)
dependencia_actividades.append([])

print("Dependencia entre actividades:")
print(dependencia_actividades)

datos = DatosActividades(n, m, cualif_actividades, tiempo_actividades, dependencia_actividades)

problema_actividades = ProblemaGenetico(decodifica_actividades, muta_actividades, fun_cruza_actividades, fitness_actividades, datos)


Tiempo realización actividades:
[[59, 87, 61, 94, 75, 54, 82, 94], [91, 59, 100, 82, 93, 65, 83, 54], [84, 89, 52, 86, 93, 96, 67, 53]]
Cualificación para realización actividades:
[[0, 1], [1], [0], [0, 1], [2], [2], [2], [2]]
Dependencia entre actividades:
[[2, 5, 3, 4, 6], [2, 6, 3, 5, 4], [4, 5, 6, 3], [5], [], [], [], []]


In [36]:
algoritmo_genetico_actividades(problema_actividades,3,min,100,50,0.8,0.05)

([0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 6, 7, 3, 5, 4], 395)

In [37]:
for i in range(50):
    solucion = algoritmo_genetico_actividades(problema_actividades,3,min,100,50,0.8,0.05)
    print(solucion)

([0, 1, 0, 0, 2, 2, 2, 2, 0, 1, 2, 6, 3, 5, 4, 7], 448)
([0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 4, 6, 3, 7, 5], 422)
([0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 4, 7, 6, 3, 5], 369)
([0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 2, 4, 7, 6, 3, 5], 401)
([0, 1, 0, 0, 2, 2, 2, 2, 0, 1, 2, 4, 6, 7, 3, 5], 422)
([1, 1, 0, 0, 2, 2, 2, 2, 1, 7, 0, 2, 6, 4, 3, 5], 462)
([0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 2, 3, 4, 5, 7, 6], 461)
([0, 1, 0, 0, 2, 2, 2, 2, 1, 0, 2, 4, 7, 3, 5, 6], 401)
([0, 1, 0, 0, 2, 2, 2, 2, 1, 0, 2, 4, 7, 3, 6, 5], 401)
([0, 1, 0, 0, 2, 2, 2, 2, 0, 1, 2, 4, 7, 6, 3, 5], 369)
([0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 4, 3, 6, 5, 7], 422)
([0, 1, 0, 0, 2, 2, 2, 2, 1, 0, 2, 6, 7, 3, 4, 5], 427)
([0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 7, 3, 5, 6, 4], 462)
([0, 1, 0, 1, 2, 2, 2, 2, 1, 0, 2, 4, 7, 3, 6, 5], 401)
([0, 1, 0, 0, 2, 2, 2, 2, 0, 1, 2, 3, 4, 7, 6, 5], 429)
([1, 1, 0, 0, 2, 2, 2, 2, 1, 7, 0, 2, 3, 6, 4, 5], 462)
([0, 1, 0, 0, 2, 2, 2, 2, 0, 1, 2, 4, 7, 6, 3, 5], 369)
([0, 1, 0, 1, 2, 2, 2, 2, 0, 1, 2, 3, 7, 5, 4, 6

### Comentarios

Puesto que ya se han hecho muchas pruebas del algoritmo genético variando parámetros en los problemas de la mochila y del cuadrado mágico, no vamos a entrar de nuevo en tanto detalle. Un aspecto interesante de este problema es el cómo afecta al resultado no solo la asignación de tareas, sino también el el orden en que se realicen. Por esto, como para cada individuo de la población inicial se le asocia una ordenación válida aleatoria, tendremos individuos muy variados, que aunque coincidan en asignación (o en ordenación), pueden diferir en la ordenación (o asignación respectivamente), dando lugar a un coste distinto.